In [1]:
import tensorflow
import pandas as pd
import time
import numpy as np

# tensorflow libraries
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

# sklearn libraries are useful for preprocessing, performance measures, etc.
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# Read Data

In [2]:
df = pd.read_csv('./dataset/features_30_sec.csv')
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [3]:
df['label'].value_counts()

blues        100
classical    100
country      100
disco        100
hiphop       100
jazz         100
metal        100
pop          100
reggae       100
rock         100
Name: label, dtype: int64


Import training modules. Split and scale dataset.

In [5]:
from PartA_train import *

columns_to_drop = ['label','filename', 'length']

X_train, Y_train, X_test, Y_test = prepare_dataset(df, columns_to_drop, test_size=0.3, random_state=1)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(700, 57) (700,)
(300, 57) (300,)


# Training Data

Randomize dataset input

In [6]:
def randomize(x, y):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y.shape[0])
    shuffled_x = x[permutation, :]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y


In [7]:

X_train, Y_train =  randomize(X_train, Y_train)
X_test, Y_test = randomize (X_test, Y_test)

Import model and set model configurations

In [8]:
from PartA_model import *

model = PartA_DNN(batch_size=4, learning_rate=0.1)

model.train(X_train, Y_train, X_test, Y_test)

2021-10-06 16:24:32.960143: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-06 16:24:33.128955: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
22/22 - 2s - loss: 20.5286 - val_loss: 20.4333
Epoch 2/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 3/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 4/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 5/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 6/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 7/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 8/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 9/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 10/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 11/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 12/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 13/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 14/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 15/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 16/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 17/50
22/22 - 0s - loss: 20.5286 - val_loss: 20.4333
Epoch 

In [9]:
model.view_summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                928       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 945
Trainable params: 945
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from PartA_model import PartA_DNN

# default configuration
learning_rate = 0.001
batch_size = 1
dropout = 0.3
seed = 100
model = PartA_DNN()
num_epochs = 50
# history = model.train(X_train, Y_train)

Training the model

In [ ]:
i = 0
err = []
for i in range(num_epochs):
    z_, h_, y_, out_ = model(X_train)
    cost = loss(X_train, out)
    z, h, y , out = model(X_train)
    grad_u = -(1 * (X_train[1]) - out)
    grad_V = tf.matmul(tf.transpose(h), grad_u)
    grad_c = tf.reduce_sum(grad_u, axis=0)
    dh = h*(1-h)
    grad_z = tf.matmul(grad_u, tf.transpose(model.V))*dh
    grad_W = tf.matmul(tf.transpose(X_train), grad_z)
    grad_b = tf.reduce_sum(grad_z, axis=0)

    model.W.assign_sub(learning_rate * grad_W)
    model.b.assign_sub(learning_rate * grad_b)
    model.V.assign_sub(learning_rate * grad_V)
    model.c.assign_sub(learning_rate * grad_c)

    if i == 0:
        print('iter: {}'.format(i+1))
        print('z: {}'.format(z_))
        print('h: {}'.format(h_))
        print('y: {}'.format(y_))
        print('grad_u: {}'.format(grad_u))
        print('dh: {}'.format(dh))
        print('grad_z:{}'.format(grad_z))
        print('grad_V:{}'.format(grad_V))
        print('grad_c:{}'.format(grad_c))
        print('grad_W:{}'.format(grad_W))
        print('grad_b:{}'.format(grad_b))
        print('cost: {}'.format(cost))
        print('W = {}, b = {}'.format(model.W.numpy(), model.b.numpy()))
        print('V = {}, c = {}'.format(model.V.numpy(), model.c.numpy()))
        
    err.append(cost)
    if not i % 100:
        print('epoch: {}, m.s.e: {}'.format(i, err[i]))